# Agregando Datasets do SP196 em um Arquivo Consolidado

## Aviso: Comentários resumidos neste código; consulte a  [Documentação](https://github.com/DemikFR/Analise-Controle-Zoonoses-SP156/blob/main/README.md) para obter uma compreensão mais detalhada das razões por trás de cada trecho de código.

## Gerar o dataframe com os dados da Prefeitura

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
from io import StringIO
import chardet

### Criar o dataframe que conterá os dados necessários para a análise

In [2]:
df = pd.DataFrame()
dfs = []

## Função para buscar os caminhos para o download de cada dataset

In [3]:
def get_soup(url, search):
    response = requests.get(url)
    
    # Verificar se a conexão foi bem sucedida
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')

        # Retornar as páginas onde terão as de download
        return soup.find_all('a', href=True, attrs={'title': re.compile(search)}) 
    else:
        return f'Página{link["title"]} está inacessível. Código: {response.status_code}'

## Código para inserir os dados em uma lista

In [4]:
# URL base do site da Prefeitura
url_base = 'http://dados.prefeitura.sp.gov.br'

# URL inicial onde estarão os links para os datasets
url_inicial = 'http://dados.prefeitura.sp.gov.br/dataset/dados-do-sp156'

# extensão do dataset para filtrar
file_type = '.csv'

for link in get_soup(url_inicial, r'^Dados do SP156.*'):
    
    # Pegar o link de download
    for page in get_soup(url_base+link['href'], r'^http://dados.prefeitura.sp.gov.br/dataset/.*'):
        link_download = page['href']
        
        # Verificar se é o arquivo a ser baixado é Excel
        if file_type in link_download:
            
            # Pegar o conteúdo do CSV
            response_csv = requests.get(link_download)
            
            # Cada datset tem um encoding diferente, por isso foi necessário usar a biblioteca chardet para identificar 
            # automaticamente qual é.
            encoding = chardet.detect(response_csv.content)['encoding']
            
            # Colocar os dados na tabela auxiliar
            if int(link["title"][-4:]) >= 2020:
                df3 = pd.read_csv(StringIO(response_csv.content.decode(encoding)), sep=';', encoding=encoding, low_memory=False)
            else:
                df3 = pd.read_csv(StringIO(response_csv.content.decode(encoding)), sep=',', encoding=encoding, low_memory=False)
                
            dfs.append(df3)  # Adicionar o DataFrame intermediário à lista
            
            # Inserir os dados na planilha original
            print(f'Os dados do {link["title"]} foram inseridos com sucesso.')

Os dados do Dados do SP156 - 1º TRI 2023 foram inseridos com sucesso.
Os dados do Dados do SP156 - 4º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 4º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - Dezembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - Outubro e Novembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º SEM 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º SEM 2019 foram inseridos com sucesso.
Os

## Criar o dataframe com todos os datasets coletados

In [5]:
df = pd.concat(dfs, ignore_index=True)

In [6]:
df.shape

(14183650, 33)

## Verificar todas as colunas disponibilizadas

In [7]:
df.any()

Data de abertura                                                          True
Canal                                                                     True
Tema                                                                      True
Assunto                                                                   True
Serviço                                                                   True
Logradouro                                                                True
Número                                                                    True
CEP                                                                       True
Subprefeitura                                                             True
Distrito                                                                  True
Setor                                                                     True
Quadra                                                                    True
Latitude                                            

## Manter apenas os campos necessários

In [8]:
df = df[['Data de abertura', 'Tema', 'Assunto', 'Especificação do assunto', 'Serviço', 'Bairro', 'Distrito'
         , 'Status da solicitação', 'Data do parecer']]

## Apagar registros que por terem muitos valores nulo, não servirão para a análise

In [9]:
df = df.dropna(thresh=4)

In [10]:
df[df['Data de abertura'].isnull()]

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer


In [11]:
df.sample(10)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
11570275,2015-02-11 11:53:46.000,Rua e bairro,"Calçadas, guias e postes",NaN,Calçada particular - denunciar calçada danific...,NaN,RAPOSO TAVARES,FINALIZADA,2018-08-29 19:14:09.000
1244212,2022-08-13 02:03:00,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Acolhimento para pessoas em situação de rua - ...,NaN,BELEM,FINALIZADA,2022-08-13 05:45:52
1886790,2022-06-20 11:43:31,Animais,Registro de animais - RGA,NaN,Registro Geral do Animal (RGA) - Solicitar ide...,NaN,NaN,FINALIZADA,2022-06-29 18:39:03
12548010,2014-05-16,NaN,Dengue (Mosquito Aedes aegypti),Reclamação de presença de mosquito incomodando...,NaN,VILA CARMOSINA,NaN,REALIZADA,2014-08-29 00:00:00.000
7186144,2020-01-31 11:37:05,Rua e bairro,"Reparos em asfalto, pontes e viadutos",NaN,Tapa-buraco,NaN,BRASILANDIA,INDEFERIDO,2020-04-15 17:53:53
1033483,2022-12-30 21:49:04,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Pessoa em situação de rua ? Solicitar atendime...,NaN,SE,FINALIZADA,2022-12-31 00:15:35
1708221,2022-05-13 12:19:48,Saúde,Medicamento de controle especial,NaN,Farmácias de manipulação e drogarias - Entrega...,NaN,NaN,FINALIZADA,2022-05-17 14:45:07
4236471,2021-06-15 01:16:21,Cidadania e assistência social,Renda Básica Emergencial,NaN,Renda Básica Emergencial - Consultar Situação ...,NaN,NaN,CANCELADA,2021-06-24 18:07:25
8183865,2019-11-29 13:40:06.644,Trânsito e Transporte,Sinalização e Circulação de Veículos e Pedestres,NaN,Rodízio de veículos - Isenção para pessoa com ...,NaN,NaN,INDEFERIDO,2019-12-02 19:28:48.263
6021181,2020-12-16 14:48:44,Cidadania e assistência social,Renda Básica Emergencial,NaN,Renda Básica Emergencial - Consultar benefício...,NaN,NaN,FINALIZADA,2020-12-16 14:48:50


## Filtrar apenas os dados necessários para a análise

In [12]:
df['Tema'].drop_duplicates()

0           Cidadania e assistência social
5                                  Animais
7               Obras, imóveis e habitação
10                            Rua e bairro
34                           Meio Ambiente
37                          Lixo e limpeza
41                                Urgência
54                                   Saúde
67                     Comércio e serviços
95                   Trânsito e Transporte
151                           Causa animal
188           Ouvidoria Geral do Município
227                         Pragas urbanas
431                               Finanças
439                         Acessibilidade
918                          Bilhete Único
1272                      Segurança urbana
2334                 Documentos e licenças
2885              Cultura, esporte e lazer
4389                              Educação
8048                              Trabalho
334059              Descomplica São Miguel
596920               Canais de Atendimento
597569     

### Entender o motivo de existirem nulos no 'Tema'

In [13]:
pd.set_option('display.max_rows', None)
df[df.Tema.isnull()]['Assunto'].drop_duplicates()

11883410                             Construcao de passarelas
11883411                                               Buraco
11883412                                      Poluição Sonora
11883414    Placas de Nomes de Ruas, Praças e demais Logra...
11883416                         Iluminacao - Lampada apagada
11883417             Transporte publico / Conduta de Trabalho
11883418                              Remoção veículo/carcaça
11883420                                               Elogio
11883424                            Vistoria / Infraestrutura
11883427                                                Obras
11883428                                           Edificacao
11883429                                        Animais / Cão
11883434                                         Pavimentacao
11883439                      Árvore(Calçada/Passeio Público)
11883442                          Transporte publico / Outros
11883444                                       Guias/sarjetas
11883446

### O campo 'Tema' só é usado em datasets mais recentes, por isso deve-se utilizar o campo 'Assunto' para filtrar

In [14]:
df1 = df[df['Assunto'].str.contains('Animais', regex=False) | df['Tema'].str.contains(r'^Anim.*|^Pragas.*', regex=True)].reset_index(drop=True)

In [15]:
df1.shape

(722401, 9)

In [16]:
df1.sample(20)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
70128,2022-04-09 15:43:03,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2022-04-30 15:17:43
325156,2020-02-08 14:55:13,Animais,Criação inadequada de animais,NaN,Denunciar condições inadequadas de criação,NaN,TUCURUVI,FINALIZADA,2020-03-25 13:03:21
191142,2021-04-05 12:48:14,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,CIDADE DUTRA,FINALIZADA,2021-04-09 14:36:24
467540,2016-11-23 12:48:56.200,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,SAPOPEMBA,FINALIZADA,2016-12-27 17:17:36.531
431840,2017-12-04 15:01:23.320,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Ratos - Solicitar vistoria em local infestado,NaN,AGUA RASA,FINALIZADA,2018-03-29 14:14:38.682
558112,2014-10-03,NaN,Animais / Pernilongo,Reclamação de Pernilongo,NaN,VELEIROS,NaN,REALIZADA,2014-10-07 00:00:00.000
19785,2022-10-08 16:07:23,Animais,Criação inadequada de animais,NaN,Denunciar condições inadequadas de criação,NaN,NaN,FINALIZADA,2022-12-14 16:12:20
130543,2021-10-09 20:39:36,Animais,Animal em via pública,NaN,Remoção de animal morto em via pública,NaN,JARAGUA,FINALIZADA,2021-10-13 11:50:14
153311,2021-12-12 01:59:35,Animais,Animais que podem causar doenças,NaN,Pernilongo/Mosquito - Solicitar vistoria em lo...,NaN,PIRITUBA,FINALIZADA,2022-01-07 16:48:47
122854,2022-03-15 09:05:01,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Ratos - Solicitar vistoria em local infestado,NaN,VILA GUILHERME,FINALIZADO,2022-06-07 15:19:49


## Padronizar o campo 'Serviço' com o 'Especificação do assunto'

#### Os dois campos possuem o mesmo objetivo

In [17]:
df1.loc[df1['Serviço'].isnull(), 'Serviço'] = df1.loc[df1['Serviço'].isnull(), 'Especificação do assunto']

In [18]:
df1.sample(10)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
161916,2021-07-12 17:46:59,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2021-07-19 13:50:04
175462,2021-08-18 21:41:30,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2021-08-26 13:00:31
164465,2021-07-19 14:43:39,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2021-07-27 18:05:34
100156,2022-01-11 10:48:39,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Ratos - Solicitar vistoria em local infestado,NaN,NaN,FINALIZADO,2022-01-19 15:38:48
307368,2020-08-31 14:29:53,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADA,2020-10-14 19:30:50
639594,2013-02-27,NaN,"Animais / Abelhas, Vespas, Marimbondos",Colméia/Vespeiro instalado,Colméia/Vespeiro instalado,JARDIM NORONHA,NaN,REALIZADA,2014-01-20 00:00:00.000
271226,2020-10-05 10:56:11,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,MOEMA,FINALIZADA,2020-10-21 14:23:39
263476,2020-12-07 11:47:38,Animais,Criação inadequada de animais,NaN,Denunciar condições inadequadas de criação,NaN,SANTANA,FINALIZADA,2021-01-08 14:41:08
421461,2017-08-24 17:23:05.260,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Ratos - Solicitar vistoria em local infestado,NaN,CAMPO LIMPO,FINALIZADA,2017-09-29 13:44:02.797
623773,2013-12-10,NaN,Animais / Cão,Condições de criação / maus tratos,Condições de criação / maus tratos,JARDIM SAO BENTO NOVO,NaN,REALIZADA,2013-12-27 00:00:00.000


In [19]:
df1[df1['Serviço'].isnull()]

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer


#### Deletar o campo que não terá mais utilidade

In [20]:
df1 = df1.drop('Especificação do assunto', axis=1)

## Tratamento dos campos de localização

#### Os campos 'Bairro' e 'Distrito' são ambíguos, por isso que os valores que estão nulos em 'Distrito' receberão o valor de 'Bairro', caso exista. Além disso, o campo 'Distrito' existe alguns que são número, indicando um distrito que está na metadado disponibilizado neste repositório.

### Converter os valores que estão em código para o seu nome original

In [21]:
distrito = pd.read_csv('codigos_de_distritos.csv', sep=';')

In [22]:
distrito['Nr Distrito'] = distrito['Nr Distrito'].astype(float)

In [23]:
distrito

,Nr Distrito,Nome Distrito
0,1.0,Agua Rasa
1,2.0,Alto de Pinheiros
2,3.0,Anhanguera
3,4.0,Aricanduva
4,5.0,Artur Alvim
5,6.0,Barra Funda
6,7.0,Bela Vista
7,8.0,Belem
8,9.0,Bom Retiro
9,10.0,Bras


In [24]:
df1['Distrito'] = df1['Distrito'].replace(distrito.set_index('Nr Distrito')['Nome Distrito'])

### Preencher valores de 'Distrito' com o de 'Bairro'

In [25]:
df1.loc[df1['Distrito'].isnull() & df1['Bairro'].notnull(), 'Distrito'] = df1.loc[df1['Distrito'].isnull() & df1['Bairro'].notnull(), 'Bairro']

In [26]:
df1.sample(10)

,Data de abertura,Tema,Assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
112011,2022-02-08 20:24:09,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,NaN,FINALIZADO,2022-02-15 12:59:11
121399,2022-03-09 18:26:35,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2022-03-10 10:22:30
552831,2014-08-25,NaN,Animais / Ratos,Reclamação de ratos,JARDIM POPULAR,JARDIM POPULAR,REALIZADA,2014-09-01 00:00:00.000
686849,2012-12-28 10:14:16.990,NaN,Animais / Cão,Cão Agressor,JARDIM SAO JOAO,Guaianases,CANCELADA,2012-12-28 00:00:00.000
70723,2022-04-12 04:28:59,Animais,Animais que transmitem doenças ou risco à saúde,Escorpião - Solicitar vistoria de local com es...,NaN,NaN,FINALIZADA,2022-05-04 12:57:12
352379,2019-10-30 20:36:38.545,Animais,Animais que transmitem doenças ou risco à saúde,Pernilongo/Mosquito - Solicitar vistoria em lo...,NaN,SOCORRO,FINALIZADA,2019-11-01 17:14:56.643
263794,2020-12-07 19:56:45,Animais,Criação inadequada de animais,Denunciar condições inadequadas de criação,NaN,TUCURUVI,FINALIZADA,2021-01-04 15:14:12
913,2023-01-03 15:09:09,Animais,Registro de animais - RGA,Registro Geral do Animal (RGA) - Solicitar ide...,NaN,NaN,INDEFERIDO,2023-01-04 04:38:30
619230,2013-11-08,NaN,Animais / Cão,Condições de criação / maus tratos,JARDIM DOS MANACAS,JARDIM DOS MANACAS,REALIZADA,2014-08-18 00:00:00.000
645874,2013-04-01,NaN,Animais / Cão,Condições de criação / maus tratos,JARDIM CASABLANCA,JARDIM CASABLANCA,REALIZADA,2013-09-10 00:00:00.000


In [27]:
df1 = df1.drop('Bairro', axis=1)

In [28]:
df1.sample(10)

,Data de abertura,Tema,Assunto,Serviço,Distrito,Status da solicitação,Data do parecer
59459,2022-09-03 10:35:17,Animais,Animais que transmitem doenças ou risco à saúde,Abelhas e Vespas - Solicitar vistoria em ninhos,ANHANGUERA,FINALIZADA,2022-09-23 13:14:23
690640,2012-01-16 12:33:51.757,NaN,"Animais / Abelhas, Vespas, Marimbondos",Colméia/Vespeiro instalado,Sao Rafael,REALIZADA,2012-01-19 00:00:00.000
647276,2013-04-08,NaN,Animais / Pernilongo,Reclamação de Pernilongo,PARQUE GRAJAU,REALIZADA,2013-08-27 00:00:00.000
154166,2021-12-14 20:23:17,Animais,Animais que podem causar doenças,Ratos - Solicitar vistoria em local infestado,ARICANDUVA,FINALIZADA,2021-12-21 11:29:13
226048,2021-01-03 02:44:28,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-01-04 16:34:39
656066,2013-05-31,NaN,Animais / Cão,Atropelado ou Acidentado vivo e sem proprietário,PARQUE DO LAGO,REALIZADA,2017-05-17 12:35:23.890
414633,2018-05-14 13:37:52.893,Animais,Animais que transmitem doenças ou risco à saúde,Ratos - Solicitar vistoria em local infestado,AGUA RASA,FINALIZADA,2018-07-11 13:12:45.083
601378,2014-06-11,NaN,"Animais / Abelhas, Vespas, Marimbondos",Colméia/Vespeiro instalado,JARDIM DA SAUDE,REALIZADA,2015-06-02 00:00:00.000
549666,2014-07-25,NaN,Animais / Escorpiões ou Aranhas,Reclamação de Escorpiões ou Aranhas,PARQUE ITABERABA,REALIZADA,2014-08-01 00:00:00.000
51662,2022-08-06 16:24:43,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2022-08-16 18:10:16


In [29]:
df1['Distrito'].isnull().sum()

233978

In [30]:
df1['Distrito'].notnull().sum()

488423

## Tratamento das descrições (Assunto, Serviço)

In [31]:
df2 = df1.rename({'Assunto': 'Tipo de Serviço'}, axis=1)

In [32]:
df2.sample()

,Data de abertura,Tema,Tipo de Serviço,Serviço,Distrito,Status da solicitação,Data do parecer
356122,2019-11-30 10:49:48.765,Animais,Animal em via pública,Avaliação de animal em sofrimento sem propriet...,VILA FORMOSA,FINALIZADA,2020-04-13 19:57:45.119


In [33]:
df2['Tipo de Serviço'].drop_duplicates()

0                                Registro de animais - RGA
1                              Exames, vacinas e castração
3                                        Adoção de animais
4                            Criação inadequada de animais
6                                       Animais silvestres
9                                    Animal em via pública
14         Animais que transmitem doenças ou risco à saúde
23                            Animal agressor e/ou invasor
66        Dengue/chikungunya/zika (mosquito aedes aegypti)
26265                                  Comércio de animais
128038                    Animais que podem causar doenças
361615                                      Animais/Aranha
386661                                  Animais/Carrapatos
392961                                      Animais/Pulgas
397007                                      Animais/barata
547255                                       Animais / Cão
547259                                      Animais / Ga

In [34]:
df2[['Tipo de Serviço', 'Serviço']].drop_duplicates()

,Tipo de Serviço,Serviço
0,Registro de animais - RGA,Avisar sobre animal encontrado com Registro G...
1,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente
3,Adoção de animais,Adotar cães e gatos
4,Criação inadequada de animais,Denunciar condições inadequadas de criação
6,Animais silvestres,Animais silvestres feridos ? Solicitar atendim...
9,Animal em via pública,Solicitar remoção de animal morto em via pública
12,Animal em via pública,Avaliação de animal em sofrimento sem propriet...
14,Animais que transmitem doenças ou risco à saúde,Pernilongo/Mosquito - Solicitar vistoria em lo...
15,Animais que transmitem doenças ou risco à saúde,Morcegos - Reclamar sobre local com morcegos
19,Animais que transmitem doenças ou risco à saúde,Ratos - Solicitar vistoria em local infestado


### Padronizar o Tipo de Serviço

In [35]:
df2.loc[df2['Tipo de Serviço'].str.contains('Animais[ |/]') & df2['Tipo de Serviço']
        .str.match('^(?!.*(Cão|Gato|Cavalo|silvestres|RGA|agressor|via))'), 'Tipo de Serviço'
       ] = 'Animais que transmitem doenças ou risco à saúde'

In [36]:
df2[['Tipo de Serviço', 'Serviço']].drop_duplicates()

,Tipo de Serviço,Serviço
0,Registro de animais - RGA,Avisar sobre animal encontrado com Registro G...
1,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente
3,Adoção de animais,Adotar cães e gatos
4,Criação inadequada de animais,Denunciar condições inadequadas de criação
6,Animais silvestres,Animais silvestres feridos ? Solicitar atendim...
9,Animal em via pública,Solicitar remoção de animal morto em via pública
12,Animal em via pública,Avaliação de animal em sofrimento sem propriet...
14,Animais que transmitem doenças ou risco à saúde,Pernilongo/Mosquito - Solicitar vistoria em lo...
15,Animais que transmitem doenças ou risco à saúde,Morcegos - Reclamar sobre local com morcegos
19,Animais que transmitem doenças ou risco à saúde,Ratos - Solicitar vistoria em local infestado


In [37]:
df2['Tipo de Serviço'] = df2['Tipo de Serviço'].str.replace('Animais / ', '')

In [38]:
df2['Tipo de Serviço'] = df2['Tipo de Serviço'].str.replace(' / Animais', '')

In [39]:
df2['Tipo de Serviço'] = df2['Tipo de Serviço'].str.replace('Dengue/chikungunya/zika (mosquito aedes aegypti)', 'Vistoria')

In [40]:
df2['Tipo de Serviço'].drop_duplicates()

0                               Registro de animais - RGA
1                             Exames, vacinas e castração
3                                       Adoção de animais
4                           Criação inadequada de animais
6                                      Animais silvestres
9                                   Animal em via pública
14        Animais que transmitem doenças ou risco à saúde
23                           Animal agressor e/ou invasor
66                                               Vistoria
26265                                 Comércio de animais
547255                                                Cão
547259                                               Gato
547267                                            Animais
547806                                             Cavalo
Name: Tipo de Serviço, dtype: object

### Apagar os registros de teste

In [41]:
df2 = df2.drop(df2.loc[df1['Serviço'].str.contains('TESTE', regex=True)].index)

### Padronizar o Serviço

#### Tratar o Tipo de Serviço 'Animais que transmitem doenças ou risco à saúde'

In [42]:
df2.loc[df2['Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

21          Abelhas e Vespas - Solicitar vistoria em ninhos
547261                           Colméia/Vespeiro instalado
69        Escorpião - Solicitar vistoria de local com es...
15             Morcegos - Reclamar sobre local com morcegos
547337                   Ocorrências / Condições de criação
547278                              Ocorrências com morcego
603218                               Ocorrências rotineiras
14        Pernilongo/Mosquito - Solicitar vistoria em lo...
227          Pombos - Solicitar vistoria em local infestado
19            Ratos - Solicitar vistoria em local infestado
361615                                Reclamação de Aranhas
547513                                Reclamação de Baratas
547304                             Reclamação de Carrapatos
558700                             Reclamação de Escorpiões
547285                  Reclamação de Escorpiões ou Aranhas
547455                               Reclamação de Formigas
547766                                 R

In [43]:
df3 = df2.copy()

#### Juntar Serviços iguais com nomes diferentes

In [44]:
df3.loc[(df3['Serviço'].str.contains('Pernilongo/Mosquito', regex=True)) | (df3['Serviço'] == 'Reclamação de Pernilongo'), 'Serviço'] = 'Reclamação de Pernilongos e Mosquitos'

In [127]:
df3.loc[(df3['Serviço'].str.contains('Abelha')) | (df3['Serviço'] == 'Colméia/Vespeiro instalado'), 'Serviço'] = 'Abelhas, Vespas ou Marimbondos'

In [46]:
df3.loc[df3['Serviço'].str.contains('Escorpião', regex=True), 'Serviço'] = 'Escorpiões'

In [47]:
df3.loc[df3['Serviço'] == 'Reclamação de Escorpião', 'Serviço'] = 'Reclamação de Escorpiões'

In [48]:
df3.loc[df3['Serviço'] == 'Ocorrências com morcego', 'Serviço'] = 'Reclamação de Morcegos'

#### Juntar diversos Serviços que estão duplicados, seguindo um padrão igual

In [49]:
df3.loc[df3['Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'] = df3.loc[df3[
    'Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].str.replace(r' -.*', '', regex=True)

In [50]:
df3.loc[df3['Serviço'].str.split().str.len() == 1, 'Serviço'] = 'Reclamação de ' + df3['Serviço']

In [51]:
df3.loc[df3['Tipo de Serviço'] == 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

21                                 Abelhas e Vespas
547337           Ocorrências / Condições de criação
603218                       Ocorrências rotineiras
361615                        Reclamação de Aranhas
547513                        Reclamação de Baratas
547304                     Reclamação de Carrapatos
69                         Reclamação de Escorpiões
547285          Reclamação de Escorpiões ou Aranhas
547455                       Reclamação de Formigas
15                           Reclamação de Morcegos
547766                         Reclamação de Moscas
14            Reclamação de Pernilongos e Mosquitos
227                            Reclamação de Pombos
547523                         Reclamação de Pulgas
19                              Reclamação de Ratos
397007                        Reclamação de baratas
386661                     Reclamação de carrapatos
392961                         Reclamação de pulgas
547260                          Reclamação de ratos
547261    Re

#### Tratar os Tipos de Serviços que NÃO são 'Animais que transmitem doenças ou risco à saúde'

#### Retirar caracteres não úteis

In [52]:
df3['Serviço'] = df3['Serviço'].str.replace('?', '-')
df3['Serviço'] = df3['Serviço'].str.replace('–', '-')
df3['Serviço'] = df3['Serviço'].str.replace('Invadiu o local ', 'Invadiu o local')

In [53]:
pd.set_option('display.max_colwidth', None)
df3.loc[df3['Tipo de Serviço'] != 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

547447                                                          Acidentado sem proprietário
3                                                                       Adotar cães e gatos
1460                                        Animais silvestres - Agendar entrega voluntária
6                            Animais silvestres feridos - Solicitar atendimento veterinário
109                                        Animal de grande porte solto - Solicitar remoção
547262                                     Atropelado ou Acidentado vivo e sem proprietário
547279                                                          Atropelado sem proprietário
12        Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)
0                        Avisar  sobre animal encontrado com Registro Geral do Animal (RGA)
23                                            Avisar sobre animal agressor em vias públicas
34365                  Cadastro Municipal de Comércio de Animais (CMCA) - Cancel

#### Padronizar os 'Serviços' que contém 'Acidentado' ou 'Atropelado'

In [54]:
df3.loc[df3['Serviço'].str.contains('Atropelado') | df3['Serviço'].str.contains('Acidentado'), 'Serviço'
       ] = 'Atropelado ou Acidentado vivo e sem proprietário'

#### Padronizar 'Condições de criação' e 'Denunciar condições inadequadas de criação' com 'Condições de criação / maus tratos'

In [55]:
df3.loc[df3['Serviço'].str.match('.*ondições'), 'Serviço'
       ] = 'Condições de criação / maus tratos'

#### Padronizar os que tem 'Remoção de animal morto em via pública' (3)

In [56]:
df3.loc[df3['Serviço'].str.match('.*morto em via.*'), 'Serviço'
       ] = 'Remoção de animal morto em via pública'

#### Padronizar "Castrar Cães e Gatos"

In [83]:
df3.loc[df3['Serviço'] == 'Castrar cães e gatos de protetores independentes', 'Serviço'] = 'Castrar cães e gatos gratuitamente'

#### Usar o 'Tipo de Serviço' para preencher 'Serviços' não tão bem descritos, como 'Solto em via pública'

In [57]:
condicoes = (df3['Serviço'] == 'Em parques') | (df3['Serviço'] == 'Invadiu o local') | (df3['Serviço'] == 'Solto em via pública')
df3.loc[condicoes, 'Serviço'] = df3.loc[condicoes, 'Tipo de Serviço'].str.cat(df3.loc[condicoes, 'Serviço'], sep=' ')

In [58]:
condicoes = (df3['Serviço'] == 'Diversas ocorrências') | (df3['Serviço'] == 'Ocorrências rotineiras')
df3.loc[condicoes, 'Serviço'] = df3.loc[condicoes, 'Tipo de Serviço'].str.cat(df3.loc[condicoes, 'Serviço'], sep=' - ')

In [84]:
pd.set_option('display.max_colwidth', None)
df3.loc[df3['Tipo de Serviço'] != 'Animais que transmitem doenças ou risco à saúde', 'Serviço'].drop_duplicates().sort_values()

3                                                                       Adotar cães e gatos
1460                                        Animais silvestres - Agendar entrega voluntária
6                            Animais silvestres feridos - Solicitar atendimento veterinário
109                                        Animal de grande porte solto - Solicitar remoção
547262                                     Atropelado ou acidentado vivo e sem proprietário
12        Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)
0                        Avisar  sobre animal encontrado com Registro Geral do Animal (RGA)
23                                            Avisar sobre animal agressor em vias públicas
34365                  Cadastro Municipal de Comércio de Animais (CMCA) - Cancelar cadastro
26265                    Cadastro Municipal de Comércio de Animais (CMCA) - Fazer inscrição
1                                                        Castrar cães e gatos gr

In [85]:
df3[['Tipo de Serviço', 'Serviço']].drop_duplicates().sort_values(by=['Tipo de Serviço', 'Serviço'])

,Tipo de Serviço,Serviço
3,Adoção de animais,Adotar cães e gatos
547267,Animais,Remoção de animal morto em via pública
21,Animais que transmitem doenças ou risco à saúde,Abelhas e vespas
547261,Animais que transmitem doenças ou risco à saúde,"Abelhas, Vespas ou Marimbondos"
603218,Animais que transmitem doenças ou risco à saúde,Animais que transmitem doenças ou risco à saúde - Ocorrências rotineiras
547337,Animais que transmitem doenças ou risco à saúde,Condições de criação / Maus tratos
361615,Animais que transmitem doenças ou risco à saúde,Reclamação de aranhas
397007,Animais que transmitem doenças ou risco à saúde,Reclamação de baratas
386661,Animais que transmitem doenças ou risco à saúde,Reclamação de carrapatos
69,Animais que transmitem doenças ou risco à saúde,Reclamação de escorpiões


In [86]:
df3.sample(10)

,Data de abertura,Tema,Tipo de Serviço,Serviço,Distrito,Status da solicitação,Data do parecer
171721,2021-08-09 13:06:27,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2021-08-11 14:34:45
691883,2012-01-21 05:24:00.993,NaN,Animais que transmitem doenças ou risco à saúde,Reclamação de ratos,Cangaiba,FINALIZADA,2013-02-04 00:00:00.000
566383,2014-11-24,NaN,Animais que transmitem doenças ou risco à saúde,"Abelhas, Vespas ou Marimbondos",VILA NOVA SAVOIA,FINALIZADA,2014-12-02 00:00:00.000
158576,2021-07-02 13:45:08,Animais,Animal agressor e/ou invasor,Avisar sobre animal agressor em vias públicas,PEDREIRA,FINALIZADA,2021-07-02 15:09:25
456351,2016-08-04 16:10:20.683,Animais,Animal em via pública,Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor),VILA CURUCA,FINALIZADA,2018-04-12 11:52:22.461
513981,2015-11-29 22:32:40.447,Animais,Animal em via pública,Remoção de animal morto em via pública,SAO RAFAEL,FINALIZADA,2018-04-12 11:59:53.855
255625,2021-03-16 15:17:58,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDO,2021-03-27 18:06:05
696839,2012-02-13 09:04:17.953,NaN,Animais,Remoção de animal morto em via pública,Vila Guilherme,FINALIZADA,2012-02-23 12:57:31.673
42434,2022-07-08 13:20:35,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2022-07-08 17:19:04
129115,2021-10-05 15:50:08,Animais,Animais que transmitem doenças ou risco à saúde,Abelhas e vespas,BUTANTA,FINALIZADA,2021-10-08 17:52:27


### Realizar um capitalize em cada "Serviço" caso gouver necessidade

In [88]:
def capitalize(servico):
    # Descobrir o sinal que a frase usa
    sinal = re.findall(r" - | / ", servico)
    
    if sinal:
        # Dividindo a frase em duas partes: antes e depois do traço ou da barra
        partes = re.split(sinal[0], servico)
        
        # Realizando o capitalize na primeira parte (antes do traço)
        parte1 = partes[0].capitalize()

        # Realizando o capitalize na segunda parte (depois do traço)
        parte2 = partes[1].capitalize()
        
        return "".join([parte1, sinal[0], parte2])
    else:
        return servico.capitalize()

In [89]:
df3['Serviço'] = df3['Serviço'].apply(lambda x: capitalize(x) if not re.search(r'RGA|CMCA', x) else x)

In [90]:
df3[['Tipo de Serviço', 'Serviço']].drop_duplicates().sort_values(by=['Tipo de Serviço', 'Serviço'])

,Tipo de Serviço,Serviço
3,Adoção de animais,Adotar cães e gatos
547267,Animais,Remoção de animal morto em via pública
21,Animais que transmitem doenças ou risco à saúde,Abelhas e vespas
547261,Animais que transmitem doenças ou risco à saúde,"Abelhas, vespas ou marimbondos"
603218,Animais que transmitem doenças ou risco à saúde,Animais que transmitem doenças ou risco à saúde - Ocorrências rotineiras
547337,Animais que transmitem doenças ou risco à saúde,Condições de criação / Maus tratos
361615,Animais que transmitem doenças ou risco à saúde,Reclamação de aranhas
397007,Animais que transmitem doenças ou risco à saúde,Reclamação de baratas
386661,Animais que transmitem doenças ou risco à saúde,Reclamação de carrapatos
69,Animais que transmitem doenças ou risco à saúde,Reclamação de escorpiões


### Tratar o "Status da solicitação"

In [128]:
df4 = df3.copy()

In [129]:
df4['Status da solicitação'].drop_duplicates().sort_values()

0         AGUARDANDO ATENDIMENTO
403                    CANCELADA
1                     FINALIZADA
547398                INDEFERIDA
4                     INDEFERIDO
Name: Status da solicitação, dtype: object

In [130]:
df4['Status da solicitação'] = df4['Status da solicitação'].str.replace('CANCELADO', 'CANCELADA')
df4['Status da solicitação'] = df4['Status da solicitação'].str.replace('FINALIZADO', 'FINALIZADA')
df4['Status da solicitação'] = df4['Status da solicitação'].str.replace('REALIZADA', 'FINALIZADA')
df4['Status da solicitação'] = df4['Status da solicitação'].str.replace('INDEFERIDO', 'INDEFERIDA')
df4['Status da solicitação'] = df4['Status da solicitação'].str.replace('ABERTA', 'AGUARDANDO ATENDIMENTO')
df4['Status da solicitação'] = df4['Status da solicitação'].str.replace('EM ANDAMENTO', 'AGUARDANDO ATENDIMENTO')

In [131]:
df4['Status da solicitação'].drop_duplicates().sort_values()

0      AGUARDANDO ATENDIMENTO
403                 CANCELADA
1                  FINALIZADA
4                  INDEFERIDA
Name: Status da solicitação, dtype: object

### Transformar todas as datas e horas para apenas data

#### 'Data de abertura'

In [132]:
df4['Data de abertura'] = df4['Data de abertura'].str.slice(0,10)

In [133]:
df4['Data de abertura'] = pd.to_datetime(df4['Data de abertura']).dt.date

In [134]:
df4['Data de abertura'].sample(10)

221461    2021-06-20
93959     2022-06-27
419995    2017-08-01
136692    2021-10-30
199389    2021-04-26
57631     2022-08-28
479527    2016-02-18
143737    2021-11-19
677472    2012-11-01
642866    2013-03-13
Name: Data de abertura, dtype: object

In [135]:
df4.loc[df4['Data do parecer'].notnull(), 'Data do parecer'] = df4.loc[
    df4['Data do parecer'].notnull(), 'Data do parecer'].str.slice(0,10)

In [136]:
df4['Data do parecer'].sample(10)

582631    2014-04-23
537365    2015-07-02
489       2023-01-13
398613    2018-12-23
96696     2022-01-18
587526    2014-07-30
279252    2020-10-29
366430    2019-03-27
382670    2018-08-21
578332    2014-02-06
Name: Data do parecer, dtype: object

In [137]:
df4.sample(10)

,Data de abertura,Tema,Tipo de Serviço,Serviço,Distrito,Status da solicitação,Data do parecer
270565,2020-10-02,Animais,Animal em via pública,Remoção de animal morto em via pública,PIRITUBA,CANCELADA,2020-10-02
222911,2021-06-23,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-06-29
529203,2015-02-24,Animais,Animais que transmitem doenças ou risco à saúde,"Abelhas, Vespas ou Marimbondos",NaN,FINALIZADA,2015-05-13
659687,2013-06-29,NaN,Cão,Cão agressor,JARDIM HELENA,CANCELADA,2013-07-02
215859,2021-06-06,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-06-11
199572,2021-04-26,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,FINALIZADA,2021-05-08
280593,2020-11-02,Animais,Animal agressor e/ou invasor,Avisar sobre animal agressor em vias públicas,LAJEADO,FINALIZADA,2020-12-19
8734,2023-01-27,Animais,"Exames, vacinas e castração",Castrar cães e gatos gratuitamente,NaN,INDEFERIDA,2023-01-27
79083,2022-05-09,Animais,Criação inadequada de animais,Condições de criação / Maus tratos,SAO MATEUS,FINALIZADA,2022-07-26
430726,2017-11-23,Animais,Animais que transmitem doenças ou risco à saúde,Reclamação de ratos,MANDAQUI,FINALIZADA,2019-02-26


In [138]:
df4.isnull().any()

Data de abertura         False
Tema                      True
Tipo de Serviço          False
Serviço                  False
Distrito                  True
Status da solicitação    False
Data do parecer           True
dtype: bool

## Gerar o arquivo com os dados necessários

In [139]:
df4.to_parquet('sp156_all_time.gzip',
              compression='gzip',
              index=False)